In [ ]:
# 顧客流失偵測

In [ ]:
## 載入套件與讀取資料

In [ ]:
### 載入套件

In [1]:
# 載入所需套件

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
'''
圖形中有中文字型的問題
參考
https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/359974/
'''
from matplotlib.font_manager import FontProperties

han_font = FontProperties(fname=r"c:/windows/fonts/msjh.ttc", size=14) # 中文字形

In [3]:
'''
設計圖形呈現的外觀風格
'''
sns.set(style="whitegrid")

In [ ]:
### 讀取資料

In [4]:
# 讀入資料檔
df = pd.read_excel('Online Retail.xlsx')

In [ ]:
### 資料清理

In [5]:
# 去除CustomerID沒有資料的紀錄
df = df.dropna(subset=['CustomerID'])

In [6]:
from datetime import datetime

# 取出2010-12-09到2011-12-09一年之間的資料
df = df[df.InvoiceDate>=datetime(2010, 12, 9, 0, 0, 0)]

In [7]:
#取出購買紀錄(不包含取消紀錄)
df = df[df.Quantity>0] 

In [ ]:
### 一年中的顧客購買次數

In [8]:
# 將同一個顧客的發票紀錄聚集成群後，計算不重複的發票編號數量，按照數量由大到小排序
CustomerData = df.groupby("CustomerID").agg({"InvoiceNo": "nunique"})\
.reset_index().sort_values("InvoiceNo", ascending=False)

CustomerData.head(10)

,CustomerID,InvoiceNo
322,12748.0,196
1859,14911.0,194
3960,17841.0,121
554,13089.0,94
1644,14606.0,90
2152,15311.0,88
475,12971.0,83
1672,14646.0,74
785,13408.0,60
2668,16029.0,60


In [9]:
# 一年中購買次數20次或以上的常客
MajorCustomers = CustomerData.CustomerID[CustomerData.InvoiceNo>=20].values

In [10]:
# 常客的購物紀錄
mc_df = df.loc[df.CustomerID.isin(MajorCustomers), ]

In [11]:
# 如果同一筆交易分在連續兩個時間內完成，則取最大者
mc_df = mc_df.groupby(["CustomerID", "InvoiceNo"])\
.agg({"InvoiceDate": "max"}).reset_index()

In [12]:
# 按照顧客及交易時間排列資料
mc_df = mc_df.sort_values(["CustomerID", "InvoiceDate"], ascending=True)

In [13]:
mc_df.head()

,CustomerID,InvoiceNo,InvoiceDate
0,12415.0,540267,2011-01-06 11:12:00
1,12415.0,540557,2011-01-10 09:58:00
2,12415.0,543989,2011-02-15 09:52:00
3,12415.0,545475,2011-03-03 10:59:00
4,12415.0,548661,2011-04-01 14:28:00


In [14]:
# 從最後一次購買到2011-12-10的時間
mc_todate = mc_df.groupby("CustomerID").agg({"InvoiceDate": "max"}).reset_index()\
.assign(ToDate=lambda x: datetime(2011, 12, 10, 0, 0, 0) - x.InvoiceDate)

In [31]:
# 計算每位顧客任何前後兩次購物的時間間隔
mc_freq = mc_df\
.assign(TimeDiff = mc_df.groupby("CustomerID").InvoiceDate.diff())

In [16]:
mc_freq.head(50)

,CustomerID,InvoiceNo,InvoiceDate,TimeDiff
0,12415.0,540267,2011-01-06 11:12:00,NaT
1,12415.0,540557,2011-01-10 09:58:00,3 days 22:46:00
2,12415.0,543989,2011-02-15 09:52:00,35 days 23:54:00
3,12415.0,545475,2011-03-03 10:59:00,16 days 01:07:00
4,12415.0,548661,2011-04-01 14:28:00,29 days 03:29:00
5,12415.0,553546,2011-05-17 15:42:00,46 days 01:14:00
6,12415.0,554037,2011-05-20 14:13:00,2 days 22:31:00
7,12415.0,556917,2011-06-15 13:37:00,25 days 23:24:00
8,12415.0,556918,2011-06-15 13:37:00,0 days 00:00:00
9,12415.0,559919,2011-07-13 15:30:00,28 days 01:53:00


In [32]:
# 去除TimeDiff是NaT的列
mc_freq = mc_freq.dropna(subset=["TimeDiff"])

In [33]:
# 按照顧客及交易時間間隔排列資料
mc_freq = mc_freq.sort_values(["CustomerID", "TimeDiff"], ascending=True)

In [19]:
mc_freq.head(50)

,CustomerID,InvoiceNo,InvoiceDate,TimeDiff
8,12415.0,556918,2011-06-15 13:37:00,0 days 00:00:00
10,12415.0,559920,2011-07-13 15:31:00,0 days 00:01:00
14,12415.0,565146,2011-09-01 13:51:00,0 days 00:01:00
16,12415.0,569650,2011-10-05 12:44:00,0 days 00:09:00
20,12415.0,576586,2011-11-15 14:22:00,0 days 03:50:00
18,12415.0,574138,2011-11-03 11:26:00,0 days 23:23:00
6,12415.0,554037,2011-05-20 14:13:00,2 days 22:31:00
1,12415.0,540557,2011-01-10 09:58:00,3 days 22:46:00
11,12415.0,560491,2011-07-19 10:51:00,5 days 19:20:00
19,12415.0,576394,2011-11-15 10:32:00,11 days 23:06:00


In [34]:
# 每位顧客的交易時間間隔次數
mc_intervalcount = mc_freq.groupby("CustomerID").agg({"InvoiceDate": "count"}).reset_index()\
.rename(columns={"InvoiceDate": "IntervalCount"})

In [35]:
# 將mc_freq與mc_intervalcount合併，使mc_freq取得每位顧客的交易時間間隔次數
mc_freq = mc_freq.merge(mc_intervalcount, how="left", on=["CustomerID"])

In [23]:
mc_freq.head()

,CustomerID,InvoiceNo,InvoiceDate,TimeDiff,IntervalCount
0,12415.0,556918,2011-06-15 13:37:00,00:00:00,20
1,12415.0,559920,2011-07-13 15:31:00,00:01:00,20
2,12415.0,565146,2011-09-01 13:51:00,00:01:00,20
3,12415.0,569650,2011-10-05 12:44:00,00:09:00,20
4,12415.0,576586,2011-11-15 14:22:00,03:50:00,20


In [40]:
mc_freq = mc_freq.assign(CumProb=1/mc_freq.IntervalCount)

In [41]:
mc_freq.head()

,CustomerID,InvoiceNo,InvoiceDate,TimeDiff,IntervalCount,CumProb
0,12415.0,556918,2011-06-15 13:37:00,00:00:00,20,0.05
1,12415.0,559920,2011-07-13 15:31:00,00:01:00,20,0.05
2,12415.0,565146,2011-09-01 13:51:00,00:01:00,20,0.05
3,12415.0,569650,2011-10-05 12:44:00,00:09:00,20,0.05
4,12415.0,576586,2011-11-15 14:22:00,03:50:00,20,0.05


In [43]:
mc_freq.CumProb = mc_freq.groupby("CustomerID").CumProb.cumsum()

In [44]:
mc_freq.head(50)

,CustomerID,InvoiceNo,InvoiceDate,TimeDiff,IntervalCount,CumProb
0,12415.0,556918,2011-06-15 13:37:00,0 days 00:00:00,20,0.050000
1,12415.0,559920,2011-07-13 15:31:00,0 days 00:01:00,20,0.100000
2,12415.0,565146,2011-09-01 13:51:00,0 days 00:01:00,20,0.150000
3,12415.0,569650,2011-10-05 12:44:00,0 days 00:09:00,20,0.200000
4,12415.0,576586,2011-11-15 14:22:00,0 days 03:50:00,20,0.250000
5,12415.0,574138,2011-11-03 11:26:00,0 days 23:23:00,20,0.300000
6,12415.0,554037,2011-05-20 14:13:00,2 days 22:31:00,20,0.350000
7,12415.0,540557,2011-01-10 09:58:00,3 days 22:46:00,20,0.400000
8,12415.0,560491,2011-07-19 10:51:00,5 days 19:20:00,20,0.450000
9,12415.0,576394,2011-11-15 10:32:00,11 days 23:06:00,20,0.500000
